# Demo 16: Ergodic Jump - Jump Sequence Diagnostics

Uncover hidden structure in ergodic (fully mixed, high-entropy) signals by injecting targeted harmonics to induce non-ergodic "stickiness," then peeling to extract the resonant filament.

**Like uncoiling a quantum yo-yo**: inject a harmonic into chaotic noise, then recursively peel to yank out the filament, exposing its resonant scar while collapsing noise layers.

## Key Concepts

1. **Ergodic Signal**: Fully mixed, white-noise chaotic (uniform entropy)
2. **Harmonic Injection**: Add low-frequency sine (e.g., 1/φ rhythm) to break ergodicity
3. **Ergodic Jump**: Recursive peeling extracts the injected filament
4. **Metrics**: resfrac drops to ~10⁻⁴, Hurst spikes to ~0.72 reveal structure

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import sys
sys.path.insert(0, '..')

from ergodic_jump import ErgodicJump

# Set random seed for reproducibility
np.random.seed(42)

## 1. Initialize Ergodic Jump Analyzer

Use golden ratio inverse (1/√5 ≈ 0.447) as the injection frequency.

In [ ]:
# Initialize with golden ratio frequency
jump = ErgodicJump(injection_freq=1/np.sqrt(5), amp=0.15)

print(f"Injection frequency: {jump.injection_freq:.4f}")
print(f"Amplitude: {jump.amp}")
print(f"Max peel depth: {jump.max_depth}")

## 2. Test on Pure Ergodic Signal (White Noise)

Start with truly random white noise to establish baseline behavior.

In [ ]:
# Generate pure ergodic signal
ergodic_signal = np.random.randn(1024)

# Execute ergodic jump
result = jump.execute(ergodic_signal)

print("\n" + "="*60)
print("PURE ERGODIC SIGNAL (White Noise)")
print("="*60)
print(f"\nBaseline metrics:")
print(f"  Resfrac: {result['base_resfrac']:.6f}")
print(f"  Hurst: {result['base_hurst']:.4f}")

print(f"\nAfter injection:")
print(f"  Resfrac: {result['injected_resfrac']:.6f}")
print(f"  Hurst: {result['injected_hurst']:.4f}")

print(f"\nJump metrics:")
print(f"  Resfrac drop: {result['resfrac_drop']:.6f}")
print(f"  Hurst shift: {result['hurst_shift']:.4f}")
print(f"  Filament length: {len(result['filament'])}")

if result['hurst_shift'] > 0.1:
    print(f"  ✓ Non-ergodic structure detected!")
else:
    print(f"  ~ Weak structure (expected for pure noise)")

## 3. Test on Pre-Structured Signal

Add a low-frequency sine wave to see how the jump responds to existing structure.

In [ ]:
# Generate structured signal (low-freq sine + noise)
t = np.arange(1024)
structured_signal = np.sin(2 * np.pi * 0.01 * t) + 0.3 * np.random.randn(1024)

# Execute ergodic jump
result2 = jump.execute(structured_signal)

print("\n" + "="*60)
print("PRE-STRUCTURED SIGNAL (Low-freq sine + noise)")
print("="*60)
print(f"\nBaseline metrics:")
print(f"  Resfrac: {result2['base_resfrac']:.6f}")
print(f"  Hurst: {result2['base_hurst']:.4f}")

print(f"\nJump metrics:")
print(f"  Resfrac drop: {result2['resfrac_drop']:.6f}")
print(f"  Hurst shift: {result2['hurst_shift']:.4f}")

if abs(result2['hurst_shift']) > abs(result['hurst_shift']):
    print(f"  ✓ Stronger response to injection (pre-existing structure)")

## 4. Visualize Signals and Filaments

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Plot 1: Ergodic signal
axes[0, 0].plot(ergodic_signal[:200], alpha=0.7, label='Ergodic')
axes[0, 0].set_title('Pure Ergodic Signal (White Noise)')
axes[0, 0].set_xlabel('Sample')
axes[0, 0].set_ylabel('Amplitude')
axes[0, 0].legend()
axes[0, 0].grid(True, alpha=0.3)

# Plot 2: Ergodic filament
if len(result['filament']) > 0:
    axes[0, 1].plot(result['filament'], alpha=0.7, color='red', label='Filament')
    axes[0, 1].set_title(f'Extracted Filament (Hurst shift: {result["hurst_shift"]:.3f})')
    axes[0, 1].set_xlabel('Sample')
    axes[0, 1].set_ylabel('Amplitude')
    axes[0, 1].legend()
    axes[0, 1].grid(True, alpha=0.3)

# Plot 3: Structured signal
axes[1, 0].plot(structured_signal[:200], alpha=0.7, label='Structured', color='green')
axes[1, 0].set_title('Pre-Structured Signal (Sine + Noise)')
axes[1, 0].set_xlabel('Sample')
axes[1, 0].set_ylabel('Amplitude')
axes[1, 0].legend()
axes[1, 0].grid(True, alpha=0.3)

# Plot 4: Structured filament
if len(result2['filament']) > 0:
    axes[1, 1].plot(result2['filament'], alpha=0.7, color='orange', label='Filament')
    axes[1, 1].set_title(f'Extracted Filament (Hurst shift: {result2["hurst_shift"]:.3f})')
    axes[1, 1].set_xlabel('Sample')
    axes[1, 1].set_ylabel('Amplitude')
    axes[1, 1].legend()
    axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 5. Ergodicity Diagnosis

Use the built-in diagnostic to automatically determine if a signal is ergodic.

In [ ]:
# Diagnose white noise
diag1 = jump.diagnose_ergodicity(ergodic_signal)

print("\n" + "="*60)
print("ERGODICITY DIAGNOSIS")
print("="*60)

print(f"\nWhite noise:")
print(f"  Is ergodic: {diag1['is_ergodic']}")
print(f"  Confidence: {diag1['confidence']:.2f}")
print(f"  Hurst: {diag1['hurst']:.4f}")
print(f"  Recommendation: {diag1['recommendation']}")

# Diagnose structured signal
diag2 = jump.diagnose_ergodicity(structured_signal)

print(f"\nStructured signal:")
print(f"  Is ergodic: {diag2['is_ergodic']}")
print(f"  Confidence: {diag2['confidence']:.2f}")
print(f"  Hurst: {diag2['hurst']:.4f}")
print(f"  Recommendation: {diag2['recommendation']}")

## 6. Comparison: Hurst Exponent Shifts

Visualize how the Hurst exponent changes with injection.

In [ ]:
# Compare Hurst shifts
signals = ['White Noise', 'Structured']
base_hursts = [result['base_hurst'], result2['base_hurst']]
injected_hursts = [result['injected_hurst'], result2['injected_hurst']]

x = np.arange(len(signals))
width = 0.35

fig, ax = plt.subplots(figsize=(10, 6))
bars1 = ax.bar(x - width/2, base_hursts, width, label='Baseline', alpha=0.8)
bars2 = ax.bar(x + width/2, injected_hursts, width, label='After Injection', alpha=0.8)

# Add reference line at H=0.5 (ergodic)
ax.axhline(y=0.5, color='red', linestyle='--', alpha=0.5, label='Ergodic (H=0.5)')

ax.set_ylabel('Hurst Exponent')
ax.set_title('Hurst Exponent: Baseline vs After Injection')
ax.set_xticks(x)
ax.set_xticklabels(signals)
ax.legend()
ax.grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

print(f"\nHurst shifts:")
print(f"  White noise: {result['hurst_shift']:+.4f}")
print(f"  Structured: {result2['hurst_shift']:+.4f}")

## 7. Use Case: Error Residual Analysis

Demonstrate using ergodic jump to detect hidden patterns in "random" error residuals.

In [ ]:
# Simulate error residuals with hidden bias
true_errors = np.random.randn(1024)
hidden_bias = 0.05 * np.sin(2 * np.pi * 0.005 * np.arange(1024))  # Very subtle
biased_errors = true_errors + hidden_bias

# Diagnose both
diag_true = jump.diagnose_ergodicity(true_errors)
diag_biased = jump.diagnose_ergodicity(biased_errors)

print("\n" + "="*60)
print("ERROR RESIDUAL ANALYSIS")
print("="*60)

print(f"\nTrue random errors:")
print(f"  Is ergodic: {diag_true['is_ergodic']}")
print(f"  Confidence: {diag_true['confidence']:.2f}")

print(f"\nBiased errors (hidden pattern):")
print(f"  Is ergodic: {diag_biased['is_ergodic']}")
print(f"  Confidence: {diag_biased['confidence']:.2f}")
print(f"  Hurst shift: {diag_biased['hurst_shift']:+.4f}")

if not diag_biased['is_ergodic']:
    print(f"\n  ⚠️  Hidden bias detected! {diag_biased['recommendation']}")

## Summary

**Ergodic Jump** reveals hidden structure in "random" signals:

- ✓ Hurst shift >0.1 indicates non-ergodic persistence
- ✓ Resfrac drop >0.05 suggests pullable structure
- ✓ Filament extraction enables targeted refinement
- ✓ Diagnostic mode for automatic ergodicity testing

**Applications:**
- Uncover latent biases in "random" errors
- Stress-test ergodicity in zeta clocks
- Enhance holographic refinement with filaments
- Detect hidden harmonics in residuals

🧵 **Quantum yo-yo uncoiled!** 🧵